#  PySpark II
## Exercicios em Sala
No VSCode ou Google Colab resolva os exercícios usando pyspark:


In [1]:
from pyspark.sql import SparkSession
import pyspark as ps
spark = (SparkSession 
            .builder 
            .appName("Python Spark SQL basic example") 
            .config("spark.some.config.option", "some-value") 
            .getOrCreate())

In [2]:
conf = ps.SparkConf().setMaster("yarn-client").setAppName("sparK-mer")
conf.set("spark.executor.heartbeatInterval","3600s")

1. Abra o arquivo realtor-data.csv

In [3]:
df_spark = (spark.read.format("csv")
      .option("header","true")
      .load("realtor-data.csv"))
df_spark.show(3)

+--------+---+----+--------+----------+-----------+--------+----------+--------------+--------+
|  status|bed|bath|acre_lot|      city|      state|zip_code|house_size|prev_sold_date|   price|
+--------+---+----+--------+----------+-----------+--------+----------+--------------+--------+
|for_sale|3.0| 2.0|    0.12|  Adjuntas|Puerto Rico|   601.0|     920.0|          null|105000.0|
|for_sale|4.0| 2.0|    0.08|  Adjuntas|Puerto Rico|   601.0|    1527.0|          null| 80000.0|
|for_sale|2.0| 1.0|    0.15|Juana Diaz|Puerto Rico|   795.0|     748.0|          null| 67000.0|
+--------+---+----+--------+----------+-----------+--------+----------+--------------+--------+
only showing top 3 rows



2. Salve e leia novamente o arquivo no formato parquet 


In [ ]:
df_spark.write.parquet('realtor.parquet' , mode='overwrite')

In [6]:
df_spark = spark.read.parquet("realtor-data.parquet")
df_spark.show(3)

+--------+---+----+--------+----------+-----------+--------+----------+--------------+--------+
|  status|bed|bath|acre_lot|      city|      state|zip_code|house_size|prev_sold_date|   price|
+--------+---+----+--------+----------+-----------+--------+----------+--------------+--------+
|for_sale|3.0| 2.0|    0.12|  Adjuntas|Puerto Rico|   601.0|     920.0|          null|105000.0|
|for_sale|4.0| 2.0|    0.08|  Adjuntas|Puerto Rico|   601.0|    1527.0|          null| 80000.0|
|for_sale|2.0| 1.0|    0.15|Juana Diaz|Puerto Rico|   795.0|     748.0|          null| 67000.0|
+--------+---+----+--------+----------+-----------+--------+----------+--------------+--------+
only showing top 3 rows



3. Gere um cache da tabela com 32 partições 

In [30]:
df_spark = df_spark.repartition(32)
df_spark.rdd.getNumPartitions() 

32

In [31]:
df_spark.cache()

DataFrame[status: string, bed: double, bath: double, acre_lot: double, city: string, state: string, zip_code: double, house_size: double, prev_sold_date: date, price: double]

4. Crie uma pivot table da contagem de state e status.

In [13]:
import pyspark.sql.functions as F
from pyspark.sql.functions import col, lit, when

In [9]:
df_spark.groupBy('state').pivot('status').count().show()

+--------------+--------+--------------+
|         state|for_sale|ready_to_build|
+--------------+--------+--------------+
|   Connecticut|   12178|          null|
|       Vermont|    1324|          null|
|   Puerto Rico|   24679|          null|
|Virgin Islands|    2573|          null|
|  Rhode Island|    2401|          null|
|      Virginia|       3|          null|
|    New Jersey|       2|          null|
| Massachusetts|   52513|           181|
|     Tennessee|      16|          null|
| New Hampshire|    2232|          null|
|South Carolina|      24|          null|
|      New York|    1874|          null|
+--------------+--------+--------------+



5. Mostre as 5 cidades que possuem as maiores casas (dica: use a função fillna).

In [19]:
(df_spark
 .fillna(0,subset = 'house_size')
 .groupBy('city')
 .agg(F.max('house_size').alias('house_size_max'))
 .orderBy(F.desc('house_size_max'))
 .show(5))

+------------+--------------+
|        city|house_size_max|
+------------+--------------+
|     Culebra|     1450112.0|
|  Middletown|       99999.0|
|       Acton|       99999.0|
|Frederiksted|       49658.0|
|        Moca|       43082.0|
+------------+--------------+
only showing top 5 rows



In [23]:
(df_spark
 .groupBy('state')
 .count().toPandas())

state  count
0      Connecticut  12178
1          Vermont   1324
2      Puerto Rico  24679
3   Virgin Islands   2573
4     Rhode Island   2401
5         Virginia      3
6       New Jersey      2
7    Massachusetts  52694
8    New Hampshire   2232
9        Tennessee     16
10  South Carolina     24
11        New York   1874

6. Crie a seguinte tabela, realize um inner join com essa tabela. Conte o número de registros do resultado final. 

In [25]:
import pandas as pd 
df = pd.DataFrame({'state':['Connecticut',
                            'Puerto Rico',
                            'Massachusetts',
                            'New York'],
                    'population':[3606,3264,6985,8468]})
df

state  population
0    Connecticut        3606
1    Puerto Rico        3264
2  Massachusetts        6985
3       New York        8468

In [ ]:
df_sk =spark.createDataFrame(df)
df_sk.show()

In [28]:
df_join = df_spark.join(df_sk,'state','inner')
df_join.count()